In [1]:
!nvidia-smi

Mon Jan 30 08:50:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# mount driver
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CardiacSeg/eda/Network

#install dependents
# !pip install -q "monai-weekly[nibabel, tqdm, einops]"
# !python -c "import matplotlib" || pip install -q matplotlib
# !pip install -q timm
# !pip install -U -q openmim
# !mim install -U -q mmcv-full
# !pip install ml-collections
# !pip install torchsummaryX
# %matplotlib inline

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1jz_DGnICBmKWCr_JL904PDQdIEK0_EQG/CardiacSeg/eda/Network


In [17]:
import sys

from torchsummaryX import summary

import torch

# set package path
sys.path.append('/content/drive/MyDrive/CardiacSeg')

import argparse
from networks.network import network
from networks.ssl_head import SSLHead

# sync python module
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
net = None
del net
net = network('unetcnx_x2', args)
inp = None
del inp
inp = torch.rand((1, 1, 128, 128, 128)).to(args.device)
summary(net, inp)

model: unetcnx_x2
                                                           Kernel Shape  \
Layer                                                                     
0_encoder0.layer.conv1.Conv3d_conv                     [1, 24, 3, 3, 3]   
1_encoder0.layer.InstanceNorm3d_norm1                                 -   
2_encoder0.layer.LeakyReLU_lrelu                                      -   
3_encoder0.layer.conv2.Conv3d_conv                    [24, 24, 3, 3, 3]   
4_encoder0.layer.InstanceNorm3d_norm2                                 -   
5_encoder0.layer.conv3.Conv3d_conv                     [1, 24, 1, 1, 1]   
6_encoder0.layer.InstanceNorm3d_norm3                                 -   
7_encoder0.layer.LeakyReLU_lrelu                                      -   
8_inc.Conv3d_0                                         [1, 48, 2, 2, 2]   
9_inc.Conv3d_0                                         [1, 48, 2, 2, 2]   
10_inc.LayerNorm3d_1                                               [48]   
11_inc.

/usr/local/lib/python3.8/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_encoder0.layer.conv1.Conv3d_conv,"[1, 24, 3, 3, 3]","[1, 24, 128, 128, 128]",648.0,1.358954e+09
1_encoder0.layer.InstanceNorm3d_norm1,-,"[1, 24, 128, 128, 128]",NaN,NaN
2_encoder0.layer.LeakyReLU_lrelu,-,"[1, 24, 128, 128, 128]",NaN,NaN
3_encoder0.layer.conv2.Conv3d_conv,"[24, 24, 3, 3, 3]","[1, 24, 128, 128, 128]",15552.0,3.261491e+10
4_encoder0.layer.InstanceNorm3d_norm2,-,"[1, 24, 128, 128, 128]",NaN,NaN
...,...,...,...,...
387_decoder1.decoder_block.0.LeakyReLU_lrelu,-,"[1, 24, 128, 128, 128]",NaN,NaN
388_decoder1.decoder_block.0.conv2.Conv3d_conv,"[24, 24, 3, 3, 3]","[1, 24, 128, 128, 128]",15552.0,3.261491e+10
389_decoder1.decoder_block.0.InstanceNorm3d_norm2,-,"[1, 24, 128, 128, 128]",NaN,NaN


## DenseVoxNet

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('dense_vox_net', args)
inp = torch.rand((1, 1, 96, 96, 96)).to(args.device)
summary(net, inp)

## EffcientSegNet

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('efficient_segnet_x0_2', args)
summary(net, (1, 96, 96, 96))

## UNETCNX

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
net = network('unetcnx_x0', args)
model = SSLHead(args.in_channels, net).to(args.device)
summary(model, (1, 96, 96, 96))

In [ ]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])
args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [96, 96, 96]
model = network('unetcnx_x1', args)
summary(model, (1, 96, 96, 96))